In [2]:
import ray
from ray.data.datasource.partitioning import Partitioning

root = "s3://anonymous@air-example-data/cifar-10/images"
partitioning = Partitioning("dir", field_names=["class"], base_dir=root)
dataset = ray.data.read_images(root, partitioning=partitioning)

2023-07-04 00:17:23,363	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-07-04 00:17:37,796	WARNING dataset.py:253 -- Important: Ray Data requires schemas for all datasets in Ray 2.5. This means that standalone Python objects are no longer supported. In addition, the default batch format is fixed to NumPy. To revert to legacy behavior temporarily, set the environment variable RAY_DATA_STRICT_MODE=0 on all cluster processes.

Learn more here: https://docs.ray.io/en/master/data/faq.html#migrating-to-strict-mode


In [4]:
dataset

KeyboardInterrupt: 

In [2]:
from typing import Dict

import numpy as np

CLASS_TO_LABEL = {
    "airplane": 0,
    "automobile": 1,
    "bird": 2,
    "cat": 3,
    "deer": 4,
    "dog": 5,
    "frog": 6,
    "horse": 7,
    "ship": 8,
    "truck": 9,
}

def add_label_column(batch: Dict[str, np.ndarray]) -> Dict[str, np.ndarray]:
    labels = []
    for name in batch["class"]:
        label = CLASS_TO_LABEL[name]
        labels.append(label)
    batch["label"] = np.array(labels)
    return batch

def remove_class_column(batch: Dict[str, np.ndarray]) -> Dict[str, np.ndarray]:
    del batch["class"]
    return batch

dataset = dataset.map_batches(add_label_column).map_batches(remove_class_column)

In [3]:
from torchvision import transforms

from ray.data.preprocessors import TorchVisionPreprocessor

transform = transforms.Compose([transforms.ToTensor(), transforms.CenterCrop(224)])
preprocessor = TorchVisionPreprocessor(columns=["image"], transform=transform)

per_epoch_transform = transforms.RandomHorizontalFlip(p=0.5)
per_epoch_preprocessor = TorchVisionPreprocessor(
    columns=["image"], transform=per_epoch_transform
)

In [4]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models

from ray import train
from ray.air import session
from ray.air.config import DatasetConfig, ScalingConfig
from ray.train.torch import TorchCheckpoint, TorchTrainer

def train_one_epoch(model, *, criterion, optimizer, batch_size, epoch):
    dataset_shard = session.get_dataset_shard("train")

    running_loss = 0
    for i, batch in enumerate(
        dataset_shard.iter_torch_batches(
            batch_size=batch_size, local_shuffle_buffer_size=256
        )
    ):
        inputs, labels = batch["image"], batch["label"]

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:
            session.report(
                metrics={
                    "epoch": epoch,
                    "batch": i,
                    "running_loss": running_loss / 2000,
                },
                checkpoint=TorchCheckpoint.from_model(model),
            )
            running_loss = 0

def train_loop_per_worker(config):
    model = train.torch.prepare_model(models.resnet50())
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=config["lr"])

    for epoch in range(config["epochs"]):
        train_one_epoch(
            model,
            criterion=criterion,
            optimizer=optimizer,
            batch_size=config["batch_size"],
            epoch=epoch,
        )


In [5]:
trainer = TorchTrainer(
    train_loop_per_worker=train_loop_per_worker,
    train_loop_config={"batch_size": 32, "lr": 0.02, "epochs": 1},
    datasets={"train": dataset},
    dataset_config={
        "train": DatasetConfig(per_epoch_preprocessor=per_epoch_preprocessor)
    },
    scaling_config=ScalingConfig(num_workers=2),
    preprocessor=preprocessor,
)
results = trainer.fit()

(TorchTrainer pid=955037) 2023-07-04 00:13:37,434	WARNING dataset.py:253 -- Important: Ray Data requires schemas for all datasets in Ray 2.5. This means that standalone Python objects are no longer supported. In addition, the default batch format is fixed to NumPy. To revert to legacy behavior temporarily, set the environment variable RAY_DATA_STRICT_MODE=0 on all cluster processes.
(TorchTrainer pid=955037) 
(TorchTrainer pid=955037) Learn more here: https://docs.ray.io/en/master/data/faq.html#migrating-to-strict-mode
(TorchTrainer pid=955037) 2023-07-04 00:13:39,792	INFO backend_executor.py:137 -- Starting distributed worker processes: ['955087 (172.26.215.93)', '955088 (172.26.215.93)']
(RayTrainWorker pid=955087) 2023-07-04 00:13:41,600	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=2]


(pid=955037) - RandomizeBlockOrder 1:   0%|          | 0/43 [00:00<?, ?it/s]

(pid=955037) Running 0:   0%|          | 0/43 [00:00<?, ?it/s]

(TorchTrainer pid=955037) 2023-07-04 00:13:41,708	INFO streaming_executor.py:91 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadImage->MapBatches(add_label_column)->MapBatches(remove_class_column)->TorchVisionPreprocessor] -> AllToAllOperator[RandomizeBlockOrder]
(TorchTrainer pid=955037) 2023-07-04 00:13:41,709	INFO streaming_executor.py:92 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
(TorchTrainer pid=955037) 2023-07-04 00:13:41,709	INFO streaming_executor.py:94 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


In [1]:
import datasource
data = datasource.get_data()
from sklearn.model_selection import train_test_split
import ray.data
train_indexes = []
test_indexes = []
for _, group in data.groupby('id'):
    train_index, test_index = train_test_split(group.index, test_size=0.2, shuffle=False)
    train_indexes.append(train_index)
    test_indexes.append(test_index)
ds = ray.data.from_pandas(data['FSR_for_coord'].loc[train_indexes[0]]) 

# for X in ray.data.from_pandas(data['FSR_for_coord']).iter_rows():
#     print(X)

2023-07-03 13:58:37,069	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-07-03 13:58:39,760	WARNING dataset.py:253 -- Important: Ray Data requires schemas for all datasets in Ray 2.5. This means that standalone Python objects are no longer supported. In addition, the default batch format is fixed to NumPy. To revert to legacy behavior temporarily, set the environment variable RAY_DATA_STRICT_MODE=0 on all cluster processes.

Learn more here: https://docs.ray.io/en/master/data/faq.html#migrating-to-strict-mode


In [ ]:
from trainable import Trainable
Trainable().as

In [ ]:
import ray.train.torch
ray.train.torch.TorchTrainer()

In [2]:
data['FSR_for_coord'].loc[train_indexes[0]]

Position,A,B,C,D,E,F
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.952381,0.000000,0.000000
2,0.000000,0.000000,0.000000,1.158730,0.000000,0.000000
3,0.000000,0.000000,0.000000,1.333333,0.000000,0.000000
4,0.000000,0.000000,0.000000,1.587302,0.000000,0.000000
...,...,...,...,...,...,...
446,3.571429,3.634921,5.555556,0.000000,0.904762,0.650794
447,3.682540,3.492063,5.698413,0.000000,0.920635,0.634921
448,3.761905,3.253968,5.698413,0.000000,0.952381,0.634921
449,3.777778,2.888889,5.492063,0.000000,1.015873,0.634921


In [2]:
None is None

True

In [1]:
import datasource
# data = datasource.get_data()

In [1]:
import datasource
data = datasource.get_data()
file_data = datasource.get_data()

In [4]:
file_data = datasource.get_data()

In [2]:
data

Category  id time force                             x_coord      ...  \
Position              A    B     C      D    E    F       A   B  ...   
0          0    0   0.0  0.0   0.0   22.0  0.0  0.0     NaN NaN  ...   
1          0    1   0.0  0.0   0.0  277.0  0.0  0.0     NaN NaN  ...   
2          0    2   0.0  0.0   0.0  488.0  0.0  0.0     NaN NaN  ...   
3          0    3   0.0  0.0   0.0  501.0  0.0  0.0     NaN NaN  ...   
4          0    4   0.0  0.0   0.0  540.0  0.0  0.0     NaN NaN  ...   
...       ..  ...   ...  ...   ...    ...  ...  ...     ...  ..  ...   
52144     72  635   0.0  0.0   0.0  983.0  0.0  0.0     NaN NaN  ...   
52145     72  636   0.0  0.0   0.0  962.0  0.0  0.0     NaN NaN  ...   
52146     72  637   0.0  0.0   0.0  910.0  0.0  0.0     NaN NaN  ...   
52147     72  638   0.0  0.0   0.0  851.0  0.0  0.0     NaN NaN  ...   
52148     72  639   0.0  0.0  10.0  839.0  0.0  0.0     NaN NaN  ...   

Category FSR_for_force                  FSR_for_coord                 \
Position             C      D    E    F             A    B         C   
0                  0.0    0.0  0.0  0.0           0.0  0.0  0.000000   
1                  0.0   60.0  0.0  0.0           0.0  0.0  0.000000   
2                  0.0   73.0  0.0  0.0           0.0  0.0  0.000000   
3                  0.0   84.0  0.0  0.0           0.0  0.0  0.000000   
4                  0.0  100.0  0.0  0.0           0.0  0.0  0.000000   
...                ...    ...  ...  ...           ...  ...       ...   
52144             37.0  180.0  0.0  0.0           0.0  0.0  0.587302   
52145             38.0  172.0  0.0  0.0           0.0  0.0  0.603175   
52146             38.0  165.0  0.0  0.0           0.0  0.0  0.603175   
52147             39.0  160.0  0.0  0.0           0.0  0.0  0.619048   
52148             39.0  156.0  0.0  0.0           0.0  0.0  0.619048   

Category                      
Position         D    E    F  
0         0.000000  0.0  0.0  
1         0.952381  0.0  0.0  
2         1.158730  0.0  0.0  
3         1.333333  0.0  0.0  
4         1.587302  0.0  0.0  
...            ...  ...  ...  
52144     2.857143  0.0  0.0  
52145     2.730159  0.0  0.0  
52146     2.619048  0.0  0.0  
52147     2.539683  0.0  0.0  
52148     2.476190  0.0  0.0  

[52149 rows x 32 columns]

In [13]:
import datasource
data = datasource.get_data()
train_index, test_index = datasource.get_index_splited_by_time(data)
train_dataset = datasource.FSRDataset(data['FSR_for_force'], data['force'], train_index)

In [14]:
data

Category  id time force                             x_coord      ...  \
Position              A    B     C      D    E    F       A   B  ...   
0          0    0   0.0  0.0   0.0   22.0  0.0  0.0     NaN NaN  ...   
1          0    1   0.0  0.0   0.0  277.0  0.0  0.0     NaN NaN  ...   
2          0    2   0.0  0.0   0.0  488.0  0.0  0.0     NaN NaN  ...   
3          0    3   0.0  0.0   0.0  501.0  0.0  0.0     NaN NaN  ...   
4          0    4   0.0  0.0   0.0  540.0  0.0  0.0     NaN NaN  ...   
...       ..  ...   ...  ...   ...    ...  ...  ...     ...  ..  ...   
52144     72  635   0.0  0.0   0.0  983.0  0.0  0.0     NaN NaN  ...   
52145     72  636   0.0  0.0   0.0  962.0  0.0  0.0     NaN NaN  ...   
52146     72  637   0.0  0.0   0.0  910.0  0.0  0.0     NaN NaN  ...   
52147     72  638   0.0  0.0   0.0  851.0  0.0  0.0     NaN NaN  ...   
52148     72  639   0.0  0.0  10.0  839.0  0.0  0.0     NaN NaN  ...   

Category FSR_for_force                  FSR_for_coord                 \
Position             C      D    E    F             A    B         C   
0                  0.0    0.0  0.0  0.0           0.0  0.0  0.000000   
1                  0.0   60.0  0.0  0.0           0.0  0.0  0.000000   
2                  0.0   73.0  0.0  0.0           0.0  0.0  0.000000   
3                  0.0   84.0  0.0  0.0           0.0  0.0  0.000000   
4                  0.0  100.0  0.0  0.0           0.0  0.0  0.000000   
...                ...    ...  ...  ...           ...  ...       ...   
52144             37.0  180.0  0.0  0.0           0.0  0.0  0.587302   
52145             38.0  172.0  0.0  0.0           0.0  0.0  0.603175   
52146             38.0  165.0  0.0  0.0           0.0  0.0  0.603175   
52147             39.0  160.0  0.0  0.0           0.0  0.0  0.619048   
52148             39.0  156.0  0.0  0.0           0.0  0.0  0.619048   

Category                      
Position         D    E    F  
0         0.000000  0.0  0.0  
1         0.952381  0.0  0.0  
2         1.158730  0.0  0.0  
3         1.333333  0.0  0.0  
4         1.587302  0.0  0.0  
...            ...  ...  ...  
52144     2.857143  0.0  0.0  
52145     2.730159  0.0  0.0  
52146     2.619048  0.0  0.0  
52147     2.539683  0.0  0.0  
52148     2.476190  0.0  0.0  

[52149 rows x 32 columns]

In [15]:
import pandas as pd
pd.set_option('mode.chained_assignment','raise')

In [23]:
import sklearn.preprocessing
train_indexes, test_indexes = datasource.get_index_splited_by_time(data)
scaler_X = sklearn.preprocessing.MinMaxScaler()
scaler_y = sklearn.preprocessing.MinMaxScaler()

import numpy as np
import torch
index_X = 'FSR_for_force'
index_y = 'force'
concated_train_indexes = np.concatenate(train_indexes)
scaler_X.fit(data.loc[concated_train_indexes, index_X])
scaler_y.fit(data.loc[concated_train_indexes, index_y])
data.loc[:, index_X] = scaler_X.transform(data.loc[:, index_X])
data.loc[:, index_y] = scaler_y.transform(data.loc[:, index_y])
train_dataset = datasource.FSRDataset(data.loc[:, index_X], data.loc[:, index_y], train_indexes)
test_dataset = datasource.FSRDataset(data.loc[:, index_X], data.loc[:, index_y], test_indexes)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=None)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=None)
import fsr_model
model = fsr_model.CNN_LSTM(input_size=len(data.loc[:, index_X].columns), output_size=len(data.loc[:, index_y].columns), cnn_hidden_size=4, lstm_hidden_size=4, cnn_num_layer=4, lstm_num_layer=4)
for X, y in train_loader:
    print(X.shape)
    print(y.shape)
    model(X)
    break

torch.Size([423, 6])
torch.Size([423, 6])


In [21]:
data.loc[:, index_y].ndim

2

In [22]:
index_X = 'FSR_for_force'
index_y = 'force'
data_X = data[index_X]
data_y = data[index_y]
concated_train_indexes = np.concatenate(train_indexes)
scaler_X.fit(data_X.loc[concated_train_indexes])
scaler_y.fit(data_X.loc[concated_train_indexes])
data.loc[:, index_X] = scaler_X.transform(data_X)
data.loc[:, index_y] = scaler_y.transform(data_y)
train_dataset = datasource.FSRDataset(data_X, data_y, train_indexes)
test_dataset = datasource.FSRDataset(data_X, data_y, test_indexes)

model = fsr_model.CNN_LSTM(input_size=len(data.loc[:, index_X].columns), output_size=len(data.loc[:, index_y].columns), cnn_hidden_size=4, lstm_hidden_size=4, cnn_num_layer=4, lstm_num_layer=4)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=None)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=None)
for X, y in train_loader:
    print(X.shape)
    print(y.shape)
    model(X)
    break    

torch.Size([423, 6])
torch.Size([423, 6])


In [8]:
data[[('force', 'A'), ('force', 'B')]]

Category force     
Position     A    B
0          0.0  0.0
1          0.0  0.0
2          0.0  0.0
3          0.0  0.0
4          0.0  0.0
...        ...  ...
52144      0.0  0.0
52145      0.0  0.0
52146      0.0  0.0
52147      0.0  0.0
52148      0.0  0.0

[52149 rows x 2 columns]

In [17]:
class UserException1(Exception):
    pass
class UserException2(Exception):
    pass
class UserException3(Exception):
    pass

try:
    a = int(input())
    if a == 1:
        raise UserException1
    elif a == 2:
        raise UserException2
    elif a == 3:
        raise UserException3
except Exception as e:
    if isinstance(e, UserException1) or isinstance(e, UserException2):
        print('exception1 or exception2')
    elif isinstance(e, UserException3):
        print('exception3')
    print('common exception')


In [1]:
from typing import Optional, Dict, Any
from ray.tune.search.optuna import OptunaSearch
from ray.tune.search import ConcurrencyLimiter
from ray import tune
import time
from ray.air import session
import ray
import ray.tune.search.optuna.optuna_search

In [2]:
def define_by_run_func(trial:ray.tune.search.optuna.optuna_search._OptunaTrialSuggestCaptor) -> Optional[Dict[str, Any]]:
    """Define-by-run function to create the search space.

    Ensure no actual computation takes place here. That should go into
    the trainable passed to ``Tuner()`` (in this example, that's
    ``objective``).

    For more information, see https://optuna.readthedocs.io/en/stable\
    /tutorial/10_key_features/002_configurations.html

    This function should either return None or a dict with constant values.
    """

    activation = trial.suggest_categorical("activation", ["relu", "tanh"])

    # Define-by-run allows for conditional search spaces.
    if activation == "relu":
        trial.suggest_float("size", -100, 100)
        trial.suggest_float("si/ze/width", 0, 20)
        trial.suggest_float("size/height", -100, 100)
    else:
        trial.suggest_float("si/ze/width", -1, 21)
        trial.suggest_float("size", -101, 101)
        
    # Return all constants in a dictionary.
    return {"steps": 100}

In [3]:
searcher = OptunaSearch(define_by_run_func, metric="mean_loss", mode="min")
algo = ConcurrencyLimiter(searcher, max_concurrent=4)

[I 2023-07-01 20:05:22,228] A new study created in memory with name: optuna


In [4]:
def evaluate(step, width, height, activation):
    time.sleep(0.1)
    activation_boost = 10 if activation=="relu" else 0
    return (0.1 + width * step / 100) ** (-1) + height * 0.1 + activation_boost

In [5]:
def objective(config):
    for step in range(config["steps"]):
        score = evaluate(step, config["si"]["ze"]["width"], config["size"]["height"], config["activation"])
        session.report({"iterations": step, "mean_loss": score})

In [6]:
num_samples = 10
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        search_alg=searcher,
        num_samples=num_samples,
    ),
)
results = tuner.fit()

2023-07-01 20:05:24,588	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-07-01 20:05:26,439	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


TypeError: Cannot unflatten dict due the key 'size/height' having a parent key 'size', which value is not of type <class 'dict'> (got <class 'float'>). Change the key names to resolve the conflict.

In [ ]:
print("Best hyperparameters for loss found were: ", results.get_best_result("mean_loss", "min").config)

Best hyperparameters for loss found were:  {'activation': 'tanh', 'si': {'ze': {'width': 17.423511459536684}}, 'size': {'height': -79.26626664441896}, 'steps': 100}


In [5]:
import torch
class ANN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layer, output_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.output_size = output_size
        layers = [[hidden_size, hidden_size] for i in range(num_layer)]
        layers[0][0] = input_size
        layers[-1][1] = output_size
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(layer[0], layer[1]) for layer in layers
        )
    
    def forward(self, x):
        x = self.layers(x)
        return x

In [6]:
ANN(3, 5, 3, 3)

TypeError: ANN.__init__.<locals>.<genexpr> is not a Module subclass